In [8]:
import pandas as pd
from pathlib import Path

In [ ]:
pd.set_option('display.max_columns', None)

In [10]:
data_path = Path("../data")

In [11]:
df_withdrawals = pd.read_parquet(data_path / "zrive_advertiser_withdrawals.parquet")
df_advertiser = pd.read_parquet(data_path / "zrive_dim_advertiser.parquet")
df_monthly = pd.read_parquet(data_path / "zrive_fct_monthly_snapshot_advertiser.parquet")

In [12]:
df_advertiser.head()

,advertiser_zrive_id,province_id,updated_at,advertiser_province,advertiser_group_id,min_start_contrato_date,max_start_contrato_nuevo_date,contrato_churn_date
0,6732,1,2025-02-05 01:02:08,Álava,NaN,2025-01-24,None,2025-02-04
1,4841,2,2024-08-09 13:38:43,Albacete,133.0,2023-02-24,None,2023-06-03
2,2487,3,2025-01-13 17:41:02,Alicante,NaN,2024-10-02,2024-10-02,2025-01-13
3,1771,3,2024-11-07 13:26:50,Alicante,185.0,2021-11-15,None,2024-11-06
4,3396,3,2023-11-02 13:51:07,Alicante,NaN,2023-03-17,2023-03-17,2023-11-01


In [14]:
df_monthly.head()

,advertiser_zrive_id,period_int,monthly_published_ads,monthly_unique_published_ads,monthly_contracted_ads,monthly_leads,monthly_visits,monthly_oro_ads,monthly_plata_ads,monthly_destacados_ads,monthly_pepitas_ads,monthly_shows,monthly_total_phone_views,monthly_total_calls,monthly_total_emails,monthly_total_invoice,monthly_unique_calls,monthly_unique_emails,monthly_unique_leads,monthly_avg_ad_price,monthly_distinct_ads,has_active_contract
0,1,202301,47,47,75,18,40890.0,6,6,6,0,2051941.5,14,15,0,440.8,12,3,15,NaN,NaN,True
1,2,202301,31,31,150,4,17970.0,10,10,4,0,1250403.0,16,2,2,75.4,2,2,4,NaN,NaN,True
2,3,202301,0,0,0,0,0.0,0,0,0,0,0.0,0,0,0,0.0,0,0,0,NaN,NaN,False
3,4,202301,79,79,85,14,27157.5,3,3,1,0,1142673.0,10,8,2,299.6,6,6,12,NaN,NaN,True
4,6,202301,20,20,20,16,79492.5,0,0,1,0,1773345.0,10,4,3,86.5,4,11,15,NaN,NaN,True


In [ ]:
df_withdrawals.head()

,withdrawal_id,advertiser_zrive_id,withdrawal_status,withdrawal_type,withdrawal_creation_date,withdrawal_reason
0,0,257,Cerrada,TOTAL,2012-06-19 07:12:34,RESULTADOS
1,1,219,Cerrada,TOTAL,2012-06-19 07:16:34,RESULTADOS
2,7,487,Cerrada,TOTAL,2012-06-20 07:10:16,RESULTADOS
3,12,476,Cerrada,TOTAL,2012-06-20 11:59:36,RESULTADOS
4,16,452,Cerrada,TOTAL,2012-06-20 15:41:39,FALTA DE USO/TIEMPO


In [16]:
# Withdrawals 
def add_churn(df: pd.DataFrame):
    CHURN_REASONS_EXCLUDED = [
        'Upselling-cambio de contrato',
        'Cambio a Bundle Online',
        'Cambio de Contrato/propuesta/producto'
    ]
    df["churn"] = (
        (df["withdrawal_type"] == "TOTAL") &
        (df["withdrawal_status"] != "Denegada") &
        (~df["withdrawal_reason"].isin(CHURN_REASONS_EXCLUDED))
    ).astype(int)
    return df

def convert_datetime_to_month_period(df, datetime_col, new_col, drop_original=True):
    df[new_col] = pd.to_datetime(df[datetime_col]).dt.to_period('M')
    if drop_original:
        df = df.drop(columns=[datetime_col])
    return df

def add_predict_month(df: pd.DataFrame, predict_col = "predict_month", withdrawal_col="withdrawal_month", n: int = 1):
    df[predict_col] = df[withdrawal_col] - n
    return df

In [17]:
WITHDRAWAL_COLS_TO_DROP = ["withdrawal_id", "withdrawal_status", "withdrawal_type", "withdrawal_reason", "withdrawal_month"]

df_withdrawals = convert_datetime_to_month_period(
    df_withdrawals, 
    datetime_col='withdrawal_creation_date',
    new_col='withdrawal_month',
    drop_original=True
)
df_withdrawals = add_predict_month(df_withdrawals, n=1)
df_withdrawals = add_churn(df_withdrawals)
df_withdrawals.drop(columns=WITHDRAWAL_COLS_TO_DROP, inplace=True)

In [18]:
df_withdrawals.head()

,advertiser_zrive_id,predict_month,churn
0,257,2012-05,1
1,219,2012-05,1
2,487,2012-05,1
3,476,2012-05,1
4,452,2012-05,1


In [19]:
df_monthly.columns

Index(['advertiser_zrive_id', 'period_int', 'monthly_published_ads',
       'monthly_unique_published_ads', 'monthly_contracted_ads',
       'monthly_leads', 'monthly_visits', 'monthly_oro_ads',
       'monthly_plata_ads', 'monthly_destacados_ads', 'monthly_pepitas_ads',
       'monthly_shows', 'monthly_total_phone_views', 'monthly_total_calls',
       'monthly_total_emails', 'monthly_total_invoice', 'monthly_unique_calls',
       'monthly_unique_emails', 'monthly_unique_leads', 'monthly_avg_ad_price',
       'monthly_distinct_ads', 'has_active_contract'],
      dtype='object')

In [20]:
def convert_period_int_to_month_period(df, period_col='period_int', new_col='month_period'):
    """Convierte el formato YYYYMM a un período mensual de pandas"""
    df[new_col] = pd.to_datetime(df[period_col].astype(str) + '01', format='%Y%m%d').dt.to_period('M')
    return df

In [21]:
df_monthly = convert_period_int_to_month_period(df_monthly)

In [22]:
df_monthly

,advertiser_zrive_id,period_int,monthly_published_ads,monthly_unique_published_ads,monthly_contracted_ads,monthly_leads,monthly_visits,monthly_oro_ads,monthly_plata_ads,monthly_destacados_ads,monthly_pepitas_ads,monthly_shows,monthly_total_phone_views,monthly_total_calls,monthly_total_emails,monthly_total_invoice,monthly_unique_calls,monthly_unique_emails,monthly_unique_leads,monthly_avg_ad_price,monthly_distinct_ads,has_active_contract,month_period
0,1,202301,47,47,75,18,40890.0,6,6,6,0,2051941.5,14,15,0,440.8,12,3,15,NaN,NaN,True,2023-01
1,2,202301,31,31,150,4,17970.0,10,10,4,0,1250403.0,16,2,2,75.4,2,2,4,NaN,NaN,True,2023-01
2,3,202301,0,0,0,0,0.0,0,0,0,0,0.0,0,0,0,0.0,0,0,0,NaN,NaN,False,2023-01
3,4,202301,79,79,85,14,27157.5,3,3,1,0,1142673.0,10,8,2,299.6,6,6,12,NaN,NaN,True,2023-01
4,6,202301,20,20,20,16,79492.5,0,0,1,0,1773345.0,10,4,3,86.5,4,11,15,NaN,NaN,True,2023-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80122,6800,202412,7,7,10,0,2857.5,1,1,4,0,80052.0,0,0,0,0.0,0,0,0,8840.00,8.0,False,2024-12
80123,6808,202412,17,17,20,3,12967.5,0,0,0,0,245994.0,3,2,0,0.0,2,1,3,36487.70,28.0,False,2024-12
80124,6822,202412,18,18,35,60,74362.5,5,6,15,0,1404081.0,21,31,7,0.0,26,26,52,7595.83,48.0,False,2024-12
80125,6823,202412,10,10,10,4,23857.5,0,0,1,0,302799.0,1,1,0,0.0,1,3,4,49590.00,15.0,False,2024-12


In [23]:
def add_churn_target(df_monthly, df_withdrawals):
    """
    Añade el target de churn a df_monthly, usando lo calculado en df_withdrawals

    Args:
        df_monthly: DataFrame con datos mensuales de anunciantes
        df_withdrawals: DataFrame procesado con información de churn
        
    Returns:
        DataFrame con columna 'churn' añadida
    """
    # Realizar la unión basada en advertiser_id y el período mensual
    df_target = df_monthly.merge(
        df_withdrawals[['advertiser_zrive_id', 'predict_month', 'churn']],
        left_on=['advertiser_zrive_id', 'month_period'],
        right_on=['advertiser_zrive_id', 'predict_month'],
        how='left'
    )

    # Rellenar los valores NaN en churn con 0 (no hubo churn)
    df_target['churn'] = df_target['churn'].fillna(0)

    # Eliminar la columna redundante predict_month
    if 'predict_month' in df_target.columns:
        df_target = df_target.drop(columns=['predict_month'])

    return df_target

In [24]:
df_target = add_churn_target(df_monthly, df_withdrawals)
df_target['churn'] = df_target['churn'].astype(int)

In [25]:
df_target[df_target['advertiser_zrive_id'] == 1]

,advertiser_zrive_id,period_int,monthly_published_ads,monthly_unique_published_ads,monthly_contracted_ads,monthly_leads,monthly_visits,monthly_oro_ads,monthly_plata_ads,monthly_destacados_ads,monthly_pepitas_ads,monthly_shows,monthly_total_phone_views,monthly_total_calls,monthly_total_emails,monthly_total_invoice,monthly_unique_calls,monthly_unique_emails,monthly_unique_leads,monthly_avg_ad_price,monthly_distinct_ads,has_active_contract,month_period,churn
0,1,202301,47,47,75,18,40890.0,6,6,6,0,2051941.5,14,15,0,440.8,12,3,15,NaN,NaN,True,2023-01,0
3553,1,202302,44,44,50,10,34740.0,3,3,1,0,1761028.5,18,6,3,464.8,5,4,9,NaN,NaN,True,2023-02,1
6948,1,202303,39,39,50,7,11850.0,3,3,1,0,561550.5,5,4,2,464.8,3,3,6,NaN,NaN,True,2023-03,0
10383,1,202304,39,39,50,6,15930.0,3,3,1,0,867489.0,7,3,2,464.8,3,3,6,NaN,NaN,True,2023-04,0
13816,1,202305,0,0,150,8,19140.0,0,3,1,0,1038817.5,13,4,2,464.8,4,4,8,NaN,NaN,True,2023-05,0


En el id 1 el churn ocurre en 202303, por lo tanto va a tener churn = 1 en la fila 202302.

La actividad post churn no nos interesa, la eliminaremos.

Para usuarios que tienen varios contratos y varios churns, haremos lo mismo. (posible mejora para el futuro --> detectar varias churns)

In [26]:
def remove_activity_after_first_churn(df):
    df_sorted = df.sort_values(['advertiser_zrive_id', 'period_int'])

    first_churn = df_sorted[df_sorted['churn'] == 1].groupby('advertiser_zrive_id')['period_int'].min().reset_index()
    first_churn.rename(columns={'period_int': 'first_churn_period'}, inplace=True)

    df_with_churn_info = df_sorted.merge(
    first_churn,
    on='advertiser_zrive_id',
    how='left'
    )

    df_filtered = df_with_churn_info[
    (df_with_churn_info['first_churn_period'].isna()) |
    (df_with_churn_info['period_int'] <= df_with_churn_info['first_churn_period'])
    ]

    df_filtered = df_filtered.drop(columns = ['first_churn_period'])

    return df_filtered

In [27]:
df_target = remove_activity_after_first_churn(df_target)

Existen usuarios que dejan de tener actividad pero no tienen churn confirmada en df_withdrawals entre 2023 y 2024 (caso 2 en el notebook data.ipynb). A continuación vemos cuantos usuarios en cada mes

In [28]:
user_churn_status = df_target.groupby('advertiser_zrive_id')['churn'].max().reset_index()
# 1. Identificar usuarios sin churn
users_without_churn_ids = user_churn_status[user_churn_status['churn'] == 0]['advertiser_zrive_id'].tolist()

# 2. Analizar el último período registrado para cada usuario sin churn
last_period_by_user = df_target.groupby('advertiser_zrive_id')['period_int'].max().reset_index()
last_period_no_churn = last_period_by_user[last_period_by_user['advertiser_zrive_id'].isin(users_without_churn_ids)]

# 3. Distribución resumida de los últimos períodos para usuarios sin churn
latest_period = df_target['period_int'].max()
users_active_until_end = last_period_no_churn[last_period_no_churn['period_int'] == latest_period].shape[0]
early_ending_users = last_period_no_churn[last_period_no_churn['period_int'] < latest_period].shape[0]

print(f"Usuarios sin churn activos hasta el último período ({latest_period}): {users_active_until_end}")
print(f"Usuarios sin churn que terminaron antes del último período: {early_ending_users}")

# 4. Verificar el estado del contrato en el último registro para algunos usuarios que terminaron antes
early_ending_sample = last_period_no_churn[last_period_no_churn['period_int'] < latest_period]['advertiser_zrive_id'].head(5).tolist()
early_ending_details = []

for user_id in early_ending_sample:
    last_record = df_target[df_target['advertiser_zrive_id'] == user_id].sort_values('period_int').iloc[-1]
    early_ending_details.append({
        'advertiser_zrive_id': user_id,
        'last_period': last_record['period_int'],
        'has_active_contract': last_record['has_active_contract']
    })

#print("\nEjemplos de usuarios sin churn que terminaron antes:")
#print(pd.DataFrame(early_ending_details))

# 3. Distribución de los últimos períodos para usuarios sin churn
print("\n\n")
last_period_distribution = last_period_no_churn['period_int'].value_counts().sort_index()

print("Distribución de últimos períodos para usuarios sin churn:")
print(last_period_distribution)

Usuarios sin churn activos hasta el último período (202412): 2109
Usuarios sin churn que terminaron antes del último período: 563



Distribución de últimos períodos para usuarios sin churn:
period_int
202301     237
202302      85
202303      61
202304      20
202305      20
202306      11
202307      19
202308      10
202309       5
202310       7
202311      10
202312       7
202401      11
202402       5
202403       3
202404       2
202405       8
202406       8
202407       4
202408       7
202409      11
202410       8
202411       4
202412    2109
Name: count, dtype: int64


- Caso 1: Usuarios que han tenido churn a finales de 2022 y su contrato duraba hasta primer/primeros meses de 2023, ejemplo claro de esto son los 237 ids que dejan de tener actividad en 202301

- Caso 2: Usuarios que no cumplen esa condicion, ya que ocurre en meses muy posteriores a finales de 2022. Para muchos de estos casos hay una forma de saber la fecha de churn gracias a la tabla df_advertiser

In [29]:
#Ejemplo de caso 1
df_target[df_target['advertiser_zrive_id'] == 17]

,advertiser_zrive_id,period_int,monthly_published_ads,monthly_unique_published_ads,monthly_contracted_ads,monthly_leads,monthly_visits,monthly_oro_ads,monthly_plata_ads,monthly_destacados_ads,monthly_pepitas_ads,monthly_shows,monthly_total_phone_views,monthly_total_calls,monthly_total_emails,monthly_total_invoice,monthly_unique_calls,monthly_unique_emails,monthly_unique_leads,monthly_avg_ad_price,monthly_distinct_ads,has_active_contract,month_period,churn
230,17,202301,15,15,35,3,17025.0,5,5,4,0,1265271.0,4,0,1,0.0,0,3,3,NaN,NaN,True,2023-01,0
231,17,202302,14,14,35,6,18750.0,5,5,4,0,1492879.5,4,1,0,0.0,1,4,5,NaN,NaN,True,2023-02,0
232,17,202303,0,0,35,4,4080.0,0,5,4,0,290041.5,2,1,2,0.0,1,3,4,NaN,NaN,True,2023-03,0


In [30]:
df_withdrawals[df_withdrawals['advertiser_zrive_id'] == 17]

,advertiser_zrive_id,predict_month,churn
11367,17,2022-11,1


Tuvo churn en diciembre de 2022, la actividad de 202301, 202302 y 202303 era la que ya estaba contratada. Estos casos los deberiamos eliminar ya que cumplen el mismo enfoque que remove_activity_after_first_churn, simplemente el churn fue a finales de 2022 y no tenemos esa fila

In [31]:
#Ejemplo de caso 2
df_target[df_target['advertiser_zrive_id'] == 3]

,advertiser_zrive_id,period_int,monthly_published_ads,monthly_unique_published_ads,monthly_contracted_ads,monthly_leads,monthly_visits,monthly_oro_ads,monthly_plata_ads,monthly_destacados_ads,monthly_pepitas_ads,monthly_shows,monthly_total_phone_views,monthly_total_calls,monthly_total_emails,monthly_total_invoice,monthly_unique_calls,monthly_unique_emails,monthly_unique_leads,monthly_avg_ad_price,monthly_distinct_ads,has_active_contract,month_period,churn
29,3,202301,0,0,0,0,0.0,0,0,0,0,0.0,0,0,0,0.0,0,0,0,NaN,NaN,False,2023-01,0
30,3,202405,17,17,35,6,2167.5,0,0,0,0,130840.5,2,1,3,0.0,1,2,3,42864.59,17.0,True,2024-05,0
31,3,202406,17,17,35,0,1882.5,0,0,0,0,88095.0,0,0,0,0.0,0,0,0,42864.59,17.0,True,2024-06,0
32,3,202407,17,17,35,0,315.0,0,0,0,0,20601.0,0,0,0,0.0,0,0,0,42864.59,17.0,True,2024-07,0


In [32]:
df_withdrawals[df_withdrawals['advertiser_zrive_id'] == 3]

,advertiser_zrive_id,predict_month,churn
11127,3,2022-10,1


En este caso el churn es en noviembre de 2022. El usuario luego tiene un nuevo contrato que empieza en 202405, deja de tener actividad en 202407 pero no tiene churn registrada. Sin embargo, en la tabla df_advertiser si tenemos esta info para estos casos, en la columana contrato_churn_date

In [33]:
df_advertiser[df_advertiser['advertiser_zrive_id'] == 3]

,advertiser_zrive_id,province_id,updated_at,advertiser_province,advertiser_group_id,min_start_contrato_date,max_start_contrato_nuevo_date,contrato_churn_date
21,3,8,2024-08-07 13:37:09,Barcelona,41.0,2024-05-06,None,2024-07-15


In [34]:
# otro ejemplo de caso 2
df_target[df_target['advertiser_zrive_id'] == 6288]

,advertiser_zrive_id,period_int,monthly_published_ads,monthly_unique_published_ads,monthly_contracted_ads,monthly_leads,monthly_visits,monthly_oro_ads,monthly_plata_ads,monthly_destacados_ads,monthly_pepitas_ads,monthly_shows,monthly_total_phone_views,monthly_total_calls,monthly_total_emails,monthly_total_invoice,monthly_unique_calls,monthly_unique_emails,monthly_unique_leads,monthly_avg_ad_price,monthly_distinct_ads,has_active_contract,month_period,churn
79206,6288,202409,10,10,10,1,5812.5,3,3,0,0,191163.0,3,1,0,19.6,1,0,1,17863.64,11.0,True,2024-09,0
79207,6288,202410,10,10,10,5,11242.5,3,3,0,0,382504.5,4,2,1,28.3,2,2,4,17366.67,12.0,True,2024-10,0
79208,6288,202411,10,10,10,7,9840.0,0,3,0,0,260473.5,9,1,5,28.3,1,5,6,17228.57,13.0,True,2024-11,0


In [35]:
df_withdrawals[df_withdrawals['advertiser_zrive_id'] == 6288]

,advertiser_zrive_id,predict_month,churn
19286,6288,2024-08,1
19423,6288,2024-08,1


No queda la churn registrada en df_withdrawals, pero en la columna contrato_churn_date de df_advertiser si

In [36]:
df_advertiser[df_advertiser['advertiser_zrive_id'] == 6288]

,advertiser_zrive_id,province_id,updated_at,advertiser_province,advertiser_group_id,min_start_contrato_date,max_start_contrato_nuevo_date,contrato_churn_date
6092,6288,7,2024-12-01 05:36:10,Badajoz,NaN,2024-09-06,2024-09-06,2024-11-30


La idea es: 
- Para estos id que no tienen churn, buscar en la columna contrato_churn_date de la tabla df_advertiser y añadir esta info (mismo formato de antes, tiene que tener churn = 1 el mes previo a la fecha real)

- Una vez hecho este enfoque, lo que sigan teniendo churn=0 y dejen de tener actividad los eliminaremos

In [37]:
def add_churn_from_advertiser_data(df_target, df_advertiser):
    """
    Añade columna de churn desde df_advertiser
    """

    users_without_churn = df_target.groupby('advertiser_zrive_id')['churn'].max()
    users_without_churn = users_without_churn[users_without_churn == 0].index.tolist()

    churn_info = df_advertiser[
        (df_advertiser['advertiser_zrive_id'].isin(users_without_churn)) &
        (df_advertiser['contrato_churn_date'].notna())
    ][['advertiser_zrive_id', 'contrato_churn_date']]

    if churn_info.empty:
        print("No se encontraron fechas de churn adicionales en df_advertiser")
        return df_target
    
    churn_info['churn_month'] = pd.to_datetime(churn_info['contrato_churn_date']).dt.to_period('M')

    #para cada usuario con fecha de churn, encontrar el mes anterior y marcar churn=1
    updates = []
    for _, row in churn_info.iterrows():
        user_id = row['advertiser_zrive_id']
        churn_month = row['churn_month']

        churn_month_int = int(churn_month.strftime('%Y%m'))

        previous_month = pd.Period(churn_month) - 1
        previous_month_int = int(previous_month.strftime('%Y%m'))

        #verificar si existe registro para ese usuario en el mes anterior
        user_data = df_target[df_target['advertiser_zrive_id'] == user_id]
        if previous_month_int in user_data['period_int'].values:
            updates.append((user_id, previous_month_int))

    #aplicar las actualizaciones
    for user_id, period in updates:
        df_target.loc[
            (df_target['advertiser_zrive_id'] == user_id) &
            (df_target['period_int'] == period),
            'churn'
        ] = 1

    print(f"Se actualizaron {len(updates)} registros con información de churn desde df_advertiser")

    df_target = remove_activity_after_first_churn(df_target)

    return df_target

In [38]:
df_target = add_churn_from_advertiser_data(df_target, df_advertiser)

Se actualizaron 376 registros con información de churn desde df_advertiser


In [39]:
user_churn_status = df_target.groupby('advertiser_zrive_id')['churn'].max().reset_index()
# 1. Identificar usuarios sin churn
users_without_churn_ids = user_churn_status[user_churn_status['churn'] == 0]['advertiser_zrive_id'].tolist()

# 2. Analizar el último período registrado para cada usuario sin churn
last_period_by_user = df_target.groupby('advertiser_zrive_id')['period_int'].max().reset_index()
last_period_no_churn = last_period_by_user[last_period_by_user['advertiser_zrive_id'].isin(users_without_churn_ids)]

# 3. Distribución resumida de los últimos períodos para usuarios sin churn
latest_period = df_target['period_int'].max()
users_active_until_end = last_period_no_churn[last_period_no_churn['period_int'] == latest_period].shape[0]
early_ending_users = last_period_no_churn[last_period_no_churn['period_int'] < latest_period].shape[0]

print(f"Usuarios sin churn activos hasta el último período ({latest_period}): {users_active_until_end}")
print(f"Usuarios sin churn que terminaron antes del último período: {early_ending_users}")

# 4. Verificar el estado del contrato en el último registro para algunos usuarios que terminaron antes
early_ending_sample = last_period_no_churn[last_period_no_churn['period_int'] < latest_period]['advertiser_zrive_id'].head(5).tolist()
early_ending_details = []

for user_id in early_ending_sample:
    last_record = df_target[df_target['advertiser_zrive_id'] == user_id].sort_values('period_int').iloc[-1]
    early_ending_details.append({
        'advertiser_zrive_id': user_id,
        'last_period': last_record['period_int'],
        'has_active_contract': last_record['has_active_contract']
    })

#print("\nEjemplos de usuarios sin churn que terminaron antes:")
#print(pd.DataFrame(early_ending_details))

# 3. Distribución de los últimos períodos para usuarios sin churn
print("\n\n")
last_period_distribution = last_period_no_churn['period_int'].value_counts().sort_index()

print("Distribución de últimos períodos para usuarios sin churn:")
print(last_period_distribution)

Usuarios sin churn activos hasta el último período (202412): 2024
Usuarios sin churn que terminaron antes del último período: 272



Distribución de últimos períodos para usuarios sin churn:
period_int
202301     235
202302      10
202303       1
202304       1
202306       3
202307       3
202310       1
202312       1
202401       4
202402       2
202405       3
202406       3
202408       1
202409       3
202410       1
202412    2024
Name: count, dtype: int64


La mayoria de casos que quedan son de la hipotesis de caso 1. Vamos a eliminar esos usuarios y los restantes (suponemos que son fallos en los datos)

In [40]:
def remove_incomplete_users(df_target, latest_period=None):
    """
    Elimina usuarios que no tienen churn registrado y terminan antes del último período.
    
    """
    if latest_period is None:
        latest_period = df_target['period_int'].max()
    
    # Identificar usuarios sin churn
    users_without_churn = df_target.groupby('advertiser_zrive_id')['churn'].max()
    users_without_churn = users_without_churn[users_without_churn == 0].index.tolist()
    
    # Para cada usuario sin churn, obtener su último período registrado
    last_period_by_user = df_target.groupby('advertiser_zrive_id')['period_int'].max().reset_index()
    
    # Identificar usuarios sin churn que terminan antes del último período
    users_to_remove = last_period_by_user[
        (last_period_by_user['advertiser_zrive_id'].isin(users_without_churn)) & 
        (last_period_by_user['period_int'] < latest_period)
    ]['advertiser_zrive_id'].tolist()
    
    # Filtrar el DataFrame para eliminar estos usuarios
    df_filtered = df_target[~df_target['advertiser_zrive_id'].isin(users_to_remove)]
    
    print(f"Se eliminaron {len(users_to_remove)} usuarios sin churn completo")
    
    return df_filtered

In [41]:
df_target = remove_incomplete_users(df_target)

Se eliminaron 272 usuarios sin churn completo


In [42]:
df_target[df_target['advertiser_zrive_id'] == 3]

,advertiser_zrive_id,period_int,monthly_published_ads,monthly_unique_published_ads,monthly_contracted_ads,monthly_leads,monthly_visits,monthly_oro_ads,monthly_plata_ads,monthly_destacados_ads,monthly_pepitas_ads,monthly_shows,monthly_total_phone_views,monthly_total_calls,monthly_total_emails,monthly_total_invoice,monthly_unique_calls,monthly_unique_emails,monthly_unique_leads,monthly_avg_ad_price,monthly_distinct_ads,has_active_contract,month_period,churn
26,3,202301,0,0,0,0,0.0,0,0,0,0,0.0,0,0,0,0.0,0,0,0,NaN,NaN,False,2023-01,0
27,3,202405,17,17,35,6,2167.5,0,0,0,0,130840.5,2,1,3,0.0,1,2,3,42864.59,17.0,True,2024-05,0
28,3,202406,17,17,35,0,1882.5,0,0,0,0,88095.0,0,0,0,0.0,0,0,0,42864.59,17.0,True,2024-06,1


Ya tenemos info de churn en estos usuarios.

Siguiente limpieza --> Eliminar filas con has_active_contract en False y (monthly_published_ads y monthly_unique_published_ads) en 0. Las que tengan has_active_contract en False pero actividad de momento las dejamos, suponemos que son pruebas gratuitas

In [43]:
def remove_inactive_periods_without_contract(df):
    """
    Elimina filas donde has_active_contract=False y no hay anuncios publicados.
    
    Args:
        df: DataFrame con datos mensuales de anunciantes
        
    Returns:
        DataFrame filtrado
    """
    # Contar filas antes de la limpieza
    rows_before = df.shape[0]
    
    # Filtrar filas donde has_active_contract=False y no hay anuncios publicados
    df_filtered = df[~(
        (df['has_active_contract'] == False) & 
        (df['monthly_published_ads'] == 0) & 
        (df['monthly_unique_published_ads'] == 0)
    )]
    
    # Contar filas eliminadas
    rows_removed = rows_before - df_filtered.shape[0]
    
    print(f"Se eliminaron {rows_removed} filas sin contrato activo y sin actividad")
    
    return df_filtered

In [44]:
df_target = remove_inactive_periods_without_contract(df_target)

Se eliminaron 48 filas sin contrato activo y sin actividad


In [45]:
df_target[df_target['advertiser_zrive_id'] == 3]

,advertiser_zrive_id,period_int,monthly_published_ads,monthly_unique_published_ads,monthly_contracted_ads,monthly_leads,monthly_visits,monthly_oro_ads,monthly_plata_ads,monthly_destacados_ads,monthly_pepitas_ads,monthly_shows,monthly_total_phone_views,monthly_total_calls,monthly_total_emails,monthly_total_invoice,monthly_unique_calls,monthly_unique_emails,monthly_unique_leads,monthly_avg_ad_price,monthly_distinct_ads,has_active_contract,month_period,churn
27,3,202405,17,17,35,6,2167.5,0,0,0,0,130840.5,2,1,3,0.0,1,2,3,42864.59,17.0,True,2024-05,0
28,3,202406,17,17,35,0,1882.5,0,0,0,0,88095.0,0,0,0,0.0,0,0,0,42864.59,17.0,True,2024-06,1


In [46]:
df_target.shape

(56335, 24)